<a href="https://colab.research.google.com/github/Amzilynn/Drug-Potency-Predictor-AChE/blob/main/Bioactivity_Data_Concised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Introduction: A Computational Approach to Drug Discovery

## 1.1. Context: Computational Drug Discovery (CDD)

Computational Drug Discovery (CDD) represents a paradigm shift in pharmaceutical research, leveraging **cheminformatics**, **bioinformatics**, and advanced **machine learning (ML)** techniques to accelerate the identification, optimization, and validation of novel therapeutic agents. This approach significantly reduces the time, cost, and experimental resources associated with traditional wet-lab drug development.

At the heart of CDD lies **Quantitative Structure–Activity Relationship (QSAR)** modeling, which establishes a robust mathematical relationship between a compound’s structural features (*molecular descriptors*) and its biological effect (*activity*). By learning these relationships from existing experimental data, QSAR models enable reliable *in silico* prediction of bioactivity for new, untested compounds.

---

## 1.2. Therapeutic Target: Acetylcholinesterase Inhibition

The focus of this project is the prediction of molecular bioactivity against **Acetylcholinesterase (AChE)**. AChE is a critical enzyme responsible for the rapid hydrolysis of the neurotransmitter **acetylcholine (ACh)** in the synaptic cleft.

Reduced levels of acetylcholine are strongly associated with the cognitive decline observed in **Alzheimer’s disease**. As a result, compounds that inhibit AChE play a central therapeutic role by prolonging the availability of ACh and enhancing cholinergic neurotransmission.

Accurately predicting the potency of potential AChE inhibitors *in silico* is therefore a crucial early step in Alzheimer’s drug discovery, enabling efficient prioritization of promising candidates before experimental validation.

---

## 1.3. Project Objective and Pipeline Overview

The primary objective of this project is to construct a **robust, end-to-end Machine Learning pipeline** for predicting the quantitative bioactivity  
\[
\text{pIC}_{50}
\]
of chemical compounds against AChE.

This notebook integrates four key computational phases into a single, cohesive workflow:

1. **Data Acquisition and Preprocessing (Part 1)**  
   - Collection of experimental $\text{IC}_{50}$ data from the public **ChEMBL** database  
   - Data cleaning and standardization  
   - Logarithmic transformation of $\text{IC}_{50}$ values to the $\text{pIC}_{50}$ scale  

2. **Exploratory Data Analysis (EDA) (Part 2)**  
   - Analysis of the physicochemical space of the dataset  
   - Application of **Lipinski’s Rule of Five** to assess drug-likeness  

3. **Feature Engineering (Part 3)**  
   - Calculation of molecular descriptors  
   - Generation of **PubChem Fingerprints** using **PaDEL-Descriptors**, serving as numerical input features for the ML model  

4. **Model Training and Evaluation (Part 4)**  
   - Construction of a **Random Forest Regression** model for $\text{pIC}_{50}$ prediction  
   - Performance evaluation using metrics such as $R^2$, RMSE, and MAE  

This project culminates in a **validated and saved predictive model**, suitable for downstream applications such as **virtual screening** and **lead optimization** in Alzheimer’s disease drug discovery.


## **ChEMBL Database**

The [*ChEMBL Database*](https://www.ebi.ac.uk/chembl/) is a database that contains curated bioactivity data of more than 2 million compounds. It is compiled from more than 76,000 documents, 1.2 million assays and the data spans 13,000 targets and 1,800 cells and 33,000 indications.
[Data as of March 25, 2020; ChEMBL version 26].

## **Installing libraries**

Install the ChEMBL web service package so that we can retrieve bioactivity data from the ChEMBL Database.

In [1]:
! pip install chembl_webresource_client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.2/55.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.7/70.7 kB 5.2 MB/s eta 0:00:00


## **Importing libraries**

In [2]:
# Import necessary libraries
import pandas as pd
from chembl_webresource_client.new_client import new_client

## **Search for Target protein**

### **Target search for coronavirus**

In [4]:
# Target search for coronavirus
target = new_client.target
target_query = target.search('SARS-CoV-2')
targets = pd.DataFrame.from_dict(target_query)
targets

,cross_references,organism,pref_name,score,species_group_flag,target_chembl_id,target_components,target_type,tax_id
0,[],Severe acute respiratory syndrome coronavirus 2,SARS-CoV-2,33.0,False,CHEMBL4303835,[],ORGANISM,2697049.0
1,[],Severe acute respiratory syndrome-related coro...,SARS-CoV,33.0,False,CHEMBL4303836,[],ORGANISM,694009.0
2,[],Severe acute respiratory syndrome coronavirus 2,Protein cereblon-SARS-Cov-2 polyprotein,24.0,False,CHEMBL6067603,"[{'accession': 'Q96SW2', 'component_descriptio...",PROTEIN-PROTEIN INTERACTION,2697049.0
3,[],Severe acute respiratory syndrome coronavirus 2,von Hippel-Lindau disease tumor suppressor-SAR...,18.0,False,CHEMBL6067604,"[{'accession': 'P40337', 'component_descriptio...",PROTEIN-PROTEIN INTERACTION,2697049.0
4,[],Homo sapiens,Thromboxane-A synthase,15.0,False,CHEMBL1835,"[{'accession': 'P24557', 'component_descriptio...",SINGLE PROTEIN,9606.0
...,...,...,...,...,...,...,...,...,...
3376,[],Mus musculus,L-type calcium channel,0.0,False,CHEMBL3988632,"[{'accession': 'Q01815', 'component_descriptio...",PROTEIN FAMILY,10090.0
3377,[],Rattus norvegicus,Voltage-gated sodium channel,0.0,False,CHEMBL3988641,"[{'accession': 'O88457', 'component_descriptio...",PROTEIN FAMILY,10116.0
3378,[],Homo sapiens,UDP-glucuronosyltransferases (UGTs),0.0,False,CHEMBL4523985,"[{'accession': 'P22310', 'component_descriptio...",PROTEIN FAMILY,9606.0
3379,"[{'xref_id': 'CPX-3270', 'xref_name': 'IkappaB...",Mus musculus,I-kappa-B kinase,0.0,False,CHEMBL4524000,"[{'accession': 'Q60680', 'component_descriptio...",PROTEIN COMPLEX,10090.0


In [5]:
targets.shape

(3381, 9)

### **Select and retrieve bioactivity data for *SARS coronavirus 3C-like proteinase* (fifth entry)**

We will assign the fifth entry (which corresponds to the target protein, *coronavirus 3C-like proteinase*) to the ***selected_target*** variable

In [6]:
selected_target = targets.target_chembl_id[4]
selected_target

'CHEMBL1835'

Here, we will retrieve only bioactivity data for *coronavirus 3C-like proteinase* (CHEMBL3927) that are reported as IC$_{50}$ values in nM (nanomolar) unit.

In [7]:
activity = new_client.activity
res = activity.filter(target_chembl_id=selected_target).filter(standard_type="IC50")

In [8]:
df = pd.DataFrame.from_dict(res)

In [9]:
df.head(3)

,action_type,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,...,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,None,32060,[],CHEMBL808532,Inhibition of human platelet thromboxane synth...,B,None,None,BAO_0000190,...,Homo sapiens,Thromboxane-A synthase,9606,None,None,IC50,uM,UO_0000065,None,1.0
1,None,None,33196,[],CHEMBL808532,Inhibition of human platelet thromboxane synth...,B,None,None,BAO_0000190,...,Homo sapiens,Thromboxane-A synthase,9606,None,None,IC50,uM,UO_0000065,None,0.02
2,None,None,34497,[],CHEMBL808532,Inhibition of human platelet thromboxane synth...,B,None,None,BAO_0000190,...,Homo sapiens,Thromboxane-A synthase,9606,None,None,IC50,uM,UO_0000065,None,0.6


In [10]:
df.shape

(2012, 46)

Finally we will save the resulting bioactivity data to a CSV file **bioactivity_data.csv**.

In [11]:
df.to_csv('bioactivity_data_raw.csv', index=False)

In [13]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [14]:
! mkdir "/content/gdrive/My Drive/Colab Notebooks/data2"

mkdir: cannot create directory ‘/content/gdrive/My Drive/Colab Notebooks/data2’: File exists


In [15]:
! cp bioactivity_data_raw.csv "/content/gdrive/My Drive/Colab Notebooks/data2"

In [16]:
! ls -l "/content/gdrive/My Drive/Colab Notebooks/data2"

total 910
-rw------- 1 root root  13773 Jan 20  2025 bioactivity_data_preprocessed.csv
-rw------- 1 root root 917272 Dec 15 21:54 bioactivity_data_raw.csv


In [17]:
! ls


bioactivity_data_raw.csv  gdrive  sample_data


In [18]:
! head bioactivity_data_raw.csv

action_type,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,bao_label,canonical_smiles,data_validity_comment,data_validity_description,document_chembl_id,document_journal,document_year,ligand_efficiency,molecule_chembl_id,molecule_pref_name,parent_molecule_chembl_id,pchembl_value,potential_duplicate,qudt_units,record_id,relation,src_id,standard_flag,standard_relation,standard_text_value,standard_type,standard_units,standard_upper_value,standard_value,target_chembl_id,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
,,32060,[],CHEMBL808532,Inhibition of human platelet thromboxane synthase (TXA2) was determined in human platelets,B,,,BAO_0000190,BAO_0000019,assay format,CC(CCCc1cccnc1)NC(=O)c1ccc2nc3ccc(C(C)C)cc3c(=O)n2c1.Cl.Cl,,,CHEMBL1123873,J Med Chem,1987.0,"{'bei': '14.00', 'le': '0.26', 'lle': '1.49', 'sei': '7.86'}

## **Handling missing data**
If any compounds has missing value for the **standard_value** column then drop it

In [19]:
df.isnull().sum()

,0
action_type,2010
activity_comment,1091
activity_id,0
activity_properties,0
assay_chembl_id,0
assay_description,0
assay_type,0
assay_variant_accession,2012
assay_variant_mutation,2012
bao_endpoint,0


In [20]:
df2 = df[df.standard_value.notna()]
df2

,action_type,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,...,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,None,32060,[],CHEMBL808532,Inhibition of human platelet thromboxane synth...,B,None,None,BAO_0000190,...,Homo sapiens,Thromboxane-A synthase,9606,None,None,IC50,uM,UO_0000065,None,1.0
1,None,None,33196,[],CHEMBL808532,Inhibition of human platelet thromboxane synth...,B,None,None,BAO_0000190,...,Homo sapiens,Thromboxane-A synthase,9606,None,None,IC50,uM,UO_0000065,None,0.02
2,None,None,34497,[],CHEMBL808532,Inhibition of human platelet thromboxane synth...,B,None,None,BAO_0000190,...,Homo sapiens,Thromboxane-A synthase,9606,None,None,IC50,uM,UO_0000065,None,0.6
3,None,None,34499,[],CHEMBL808532,Inhibition of human platelet thromboxane synth...,B,None,None,BAO_0000190,...,Homo sapiens,Thromboxane-A synthase,9606,None,None,IC50,uM,UO_0000065,None,0.05
4,None,None,35690,[],CHEMBL808532,Inhibition of human platelet thromboxane synth...,B,None,None,BAO_0000190,...,Homo sapiens,Thromboxane-A synthase,9606,None,None,IC50,uM,UO_0000065,None,0.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2007,None,None,13337786,[],CHEMBL2393799,Inhibition of TXAS in human 1483 cells assesse...,B,None,None,BAO_0000190,...,Homo sapiens,Thromboxane-A synthase,9606,None,None,IC50,uM,UO_0000065,None,50.0
2008,None,None,14629345,[],CHEMBL3254290,Inhibition of thromboxane synthetase in intact...,B,None,None,BAO_0000190,...,Homo sapiens,Thromboxane-A synthase,9606,None,None,IC50,uM,UO_0000065,None,1.0
2009,None,None,24956906,"[{'comments': None, 'relation': None, 'result_...",CHEMBL5214476,Selectivity interaction (Eurofins-Panlabs scre...,B,None,None,BAO_0000190,...,Homo sapiens,Thromboxane-A synthase,9606,None,None,IC50,nM,UO_0000065,None,132.0
2010,"{'action_type': 'INHIBITOR', 'description': 'N...",None,24987111,[],CHEMBL5228409,Inhibition of TXA2 synthetase (unknown origin),B,None,None,BAO_0000190,...,Homo sapiens,Thromboxane-A synthase,9606,None,None,IC50,nM,UO_0000065,None,500.0


Apparently, for this dataset there is no missing data. But we can use the above code cell for bioactivity data of other target protein.

## **Data pre-processing of the bioactivity data**

### **Labeling compounds as either being active, inactive or intermediate**
The bioactivity data is in the IC50 unit. Compounds having values of less than 1000 nM will be considered to be **active** while those greater than 10,000 nM will be considered to be **inactive**. As for those values in between 1,000 and 10,000 nM will be referred to as **intermediate**.

In [21]:
bioactivity_class = []
for i in df2.standard_value:
  if float(i) <= 1000:
    bioactivity_class.append("active")
  else:
    bioactivity_class.append("inactive")

**Iterate the molecule_chembl_id to a list**

In [22]:
mol_cid =[]
for i in df2.molecule_chembl_id:
  mol_cid.append(i)

In [23]:
mol_cid

['CHEMBL1202755',
 'CHEMBL56418',
 'CHEMBL1202763',
 'CHEMBL1202758',
 'CHEMBL418452',
 'CHEMBL1204215',
 'CHEMBL56742',
 'CHEMBL267475',
 'CHEMBL1204224',
 'CHEMBL1204218',
 'CHEMBL1204223',
 'CHEMBL1204213',
 'CHEMBL57736',
 'CHEMBL60930',
 'CHEMBL300338',
 'CHEMBL58518',
 'CHEMBL57716',
 'CHEMBL1204214',
 'CHEMBL293362',
 'CHEMBL267473',
 'CHEMBL57480',
 'CHEMBL294706',
 'CHEMBL1202757',
 'CHEMBL1204216',
 'CHEMBL1202756',
 'CHEMBL57489',
 'CHEMBL60625',
 'CHEMBL301215',
 'CHEMBL60572',
 'CHEMBL70901',
 'CHEMBL174388',
 'CHEMBL362418',
 'CHEMBL341591',
 'CHEMBL427300',
 'CHEMBL172198',
 'CHEMBL425325',
 'CHEMBL175538',
 'CHEMBL267473',
 'CHEMBL2111947',
 'CHEMBL177596',
 'CHEMBL175801',
 'CHEMBL162409',
 'CHEMBL11811',
 'CHEMBL158875',
 'CHEMBL296746',
 'CHEMBL350292',
 'CHEMBL40736',
 'CHEMBL116869',
 'CHEMBL432958',
 'CHEMBL26820',
 'CHEMBL102360',
 'CHEMBL433884',
 'CHEMBL161532',
 'CHEMBL162133',
 'CHEMBL130453',
 'CHEMBL130453',
 'CHEMBL130453',
 'CHEMBL105856',
 'CHEMBL102692'

**Iterate canonical_smiles to a list**

In [24]:
canonical_smiles =[]
for i in df2.canonical_smiles:
  canonical_smiles.append(i)

In [25]:
canonical_smiles

['CC(CCCc1cccnc1)NC(=O)c1ccc2nc3ccc(C(C)C)cc3c(=O)n2c1.Cl.Cl',
 'COc1ccc2c(=O)n3cc(C(=O)NCCCCc4cccnc4)ccc3nc2c1',
 'CC(C)c1ccc2nc3ccc(C(=O)N(C)CCCCc4cccnc4)cn3c(=O)c2c1.Cl',
 'CC(C)c1ccc2nc3ccc(C(=O)NCCCCOc4cccnc4)cn3c(=O)c2c1.Cl.Cl',
 'CC(CCc1cccnc1)NC(=O)c1ccc2nc3ccc(C(C)C)cc3c(=O)n2c1',
 'Cc1cc2nc3ccc(C(=O)NCCCCc4cccnc4)cn3c(=O)c2cc1C.Cl.Cl',
 'CC(C)c1ccc2nc3ccc(C(=O)NCCCCc4ccccn4)cn3c(=O)c2c1',
 'CCCc1c(OCC(O)COc2ccc3c(=O)cc(C(=O)O)oc3c2CCC)ccc(C(C)=O)c1O',
 'CC(C)c1ccc2nc3ccc(C(=O)NCCCCc4ccncc4)cn3c(=O)c2c1.Cl.Cl',
 'CC(C)c1ccc2nc3ccc(C(=O)NCCc4cccnc4)cn3c(=O)c2c1.Cl.Cl',
 'CC(C)c1ccc2nc3ccc(C(=O)NCCCCn4ccnc4)cn3c(=O)c2c1.Cl.Cl',
 'CC(C)c1ccc2nc3ccc(C(=O)NCCCc4cccnc4)cn3c(=O)c2c1.Cl.Cl',
 'CC(C)c1ccc2nc3ccc(C(=O)NCCCCc4cccnc4)cn3c(=O)c2c1',
 'CC(C)c1ccc2nc3ccc(C(=O)NCCCCCCCc4cccnc4)cn3c(=O)c2c1',
 'Cc1ccc2nc3ccc(C(=O)NCCCCc4cccnc4)cn3c(=O)c2c1',
 'O=C(NCCCCc1cccnc1)c1ccc2nc3ccc(Br)cc3c(=O)n2c1',
 'CC(C)Oc1ccc2nc3ccc(C(=O)NCCCCc4cccnc4)cn3c(=O)c2c1',
 'CC(C)c1ccc2nc3ccc(C(=O)NCCCCC

 **Iterate standard_value to a list**

In [26]:
standard_value =[]
for i in df2.standard_value:
  standard_value.append(i)

### **Combine the 3 columns (molecule_chembl_id,canonical_smiles,standard_value) and bioactivity_class into a DataFrame**

In [27]:
selection = ['molecule_chembl_id','canonical_smiles','standard_value']
df3 = df2[selection]
df3

,molecule_chembl_id,canonical_smiles,standard_value
0,CHEMBL1202755,CC(CCCc1cccnc1)NC(=O)c1ccc2nc3ccc(C(C)C)cc3c(=...,1000.0
1,CHEMBL56418,COc1ccc2c(=O)n3cc(C(=O)NCCCCc4cccnc4)ccc3nc2c1,20.0
2,CHEMBL1202763,CC(C)c1ccc2nc3ccc(C(=O)N(C)CCCCc4cccnc4)cn3c(=...,600.0
3,CHEMBL1202758,CC(C)c1ccc2nc3ccc(C(=O)NCCCCOc4cccnc4)cn3c(=O)...,50.0
4,CHEMBL418452,CC(CCc1cccnc1)NC(=O)c1ccc2nc3ccc(C(C)C)cc3c(=O...,300.0
...,...,...,...
2007,CHEMBL2325079,O=C(N[C@H]1CCC[C@H](CO)C1)C1CCN(c2nc3cc(Cl)c(-...,50000.0
2008,CHEMBL3251056,CCCCCC(O)CO[C@@H]1[C@@H](CCCCCCC(=O)O)[C@@H]2C...,1000.0
2009,CHEMBL4516982,CCOCCn1cc(C(=O)Nc2ccc(-n3nc(-c4cccnc4)cc3C(F)(...,132.0
2010,CHEMBL280728,O=C(O)CCCCO/N=C(/c1cccnc1)c1cccc(C(F)(F)F)c1,500.0


In [28]:
bioactivity_class = pd.Series(bioactivity_class, name='bioactivity_class')
df4 = pd.concat([df3, bioactivity_class], axis=1)
df4

,molecule_chembl_id,canonical_smiles,standard_value,bioactivity_class
0,CHEMBL1202755,CC(CCCc1cccnc1)NC(=O)c1ccc2nc3ccc(C(C)C)cc3c(=...,1000.0,active
1,CHEMBL56418,COc1ccc2c(=O)n3cc(C(=O)NCCCCc4cccnc4)ccc3nc2c1,20.0,active
2,CHEMBL1202763,CC(C)c1ccc2nc3ccc(C(=O)N(C)CCCCc4cccnc4)cn3c(=...,600.0,active
3,CHEMBL1202758,CC(C)c1ccc2nc3ccc(C(=O)NCCCCOc4cccnc4)cn3c(=O)...,50.0,active
4,CHEMBL418452,CC(CCc1cccnc1)NC(=O)c1ccc2nc3ccc(C(C)C)cc3c(=O...,300.0,active
...,...,...,...,...
902,NaN,NaN,NaN,active
910,NaN,NaN,NaN,active
931,NaN,NaN,NaN,active
990,NaN,NaN,NaN,active


Saves dataframe to CSV file

In [29]:
df4.to_csv('bioactivity_data_preprocessed.csv', index=False)

In [30]:
df4.shape

(1181, 4)

In [31]:
! ls -l

total 988
-rw-r--r-- 1 root root  83676 Dec 15 21:54 bioactivity_data_preprocessed.csv
-rw-r--r-- 1 root root 917272 Dec 15 21:49 bioactivity_data_raw.csv
drwx------ 5 root root   4096 Dec 15 21:53 gdrive
drwxr-xr-x 1 root root   4096 Dec 11 14:34 sample_data


In [32]:
! cp bioactivity_data_preprocessed.csv "/content/gdrive/My Drive/Colab Notebooks/data2"

In [33]:
! ls "/content/gdrive/My Drive/Colab Notebooks/data2"

bioactivity_data_preprocessed.csv  bioactivity_data_raw.csv


---

## **Install conda and rdkit**

In [34]:
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
! conda install -c rdkit rdkit -y
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

--2025-12-15 21:54:20--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.32.241, 104.16.191.158, 2606:4700::6810:20f1, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.32.241|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85055499 (81M) [application/x-sh]
Saving to: ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’

Miniconda3-py37_4.8 100%[===================>]  81.12M   104MB/s    in 0.8s    

2025-12-15 21:54:21 (104 MB/s) - ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’ saved [85055499/85055499]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: / - done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - asn1crypto==1.3.0=py37_0
    - ca-certificates==2020.1.1=0
    - certifi==2019.11.28=py37_0
    - cffi==1.14.0=py37h2e261b9_0
    - chardet==3.0.4=py37_1003
    - conda-package-handling==1.6.0=p

In [35]:
! conda create -n rdkit_env python=3.7 -y
! conda activate rdkit_env


Solving environment: \ | failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: \ | / - \ | / - \ | / - \ | done


==> WARNING: A newer version of conda exists. <==
  current version: 23.1.0
  latest version: 25.11.1

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=25.11.1



## Package Plan ##

  environment location: /usr/local/envs/rdkit_env

  added / updated specs:
    - python=3.7


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _openmp_mutex-5.1          |            1_gnu          21 KB
    ld_impl_linux-64-2.44      |       h153f514_2         672 KB
    libffi-3.4.4               |       h6a678d5_1         141 KB
    libgcc-15.2.0              |       h69a1729_7         

In [36]:
!pip install rdkit-pypi


     |████████████████████████████████| 29.2 MB 1.3 MB/s 


In [40]:
from rdkit import Chem
print(Chem.MolFromSmiles('CCO'))


## **Load bioactivity data**

In [38]:
import pandas as pd

In [41]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data2/bioactivity_data_preprocessed.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/data2/bioactivity_data_preprocessed.csv'

In [ ]:
df.shape

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## **Calculate Lipinski descriptors**
Christopher Lipinski, a scientist at Pfizer, came up with a set of rule-of-thumb for evaluating the **druglikeness** of compounds. Such druglikeness is based on the Absorption, Distribution, Metabolism and Excretion (ADME) that is also known as the pharmacokinetic profile. Lipinski analyzed all orally active FDA-approved drugs in the formulation of what is to be known as the **Rule-of-Five** or **Lipinski's Rule**.

The Lipinski's Rule stated the following:
* Molecular weight < 500 Dalton
* Octanol-water partition coefficient (LogP) < 5
* Hydrogen bond donors < 5
* Hydrogen bond acceptors < 10

### **Import libraries**

In [ ]:
! python --version


In [ ]:
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski

### **Calculate descriptors**

In [ ]:
# Inspired by: https://codeocean.com/explore/capsules?query=tag:data-curation

def lipinski(smiles, verbose=False):

    moldata= []
    for elem in smiles:
        mol=Chem.MolFromSmiles(elem)
        moldata.append(mol)

    baseData= np.arange(1,1)
    i=0
    for mol in moldata:

        desc_MolWt = Descriptors.MolWt(mol)
        desc_MolLogP = Descriptors.MolLogP(mol)
        desc_NumHDonors = Lipinski.NumHDonors(mol)
        desc_NumHAcceptors = Lipinski.NumHAcceptors(mol)

        row = np.array([desc_MolWt,
                        desc_MolLogP,
                        desc_NumHDonors,
                        desc_NumHAcceptors])

        if(i==0):
            baseData=row
        else:
            baseData=np.vstack([baseData, row])
        i=i+1

    columnNames=["MW","LogP","NumHDonors","NumHAcceptors"]
    descriptors = pd.DataFrame(data=baseData,columns=columnNames)

    return descriptors

In [ ]:
df_lipinski = lipinski(df.canonical_smiles)

### **Combine DataFrames**

Let's take a look at the 2 DataFrames that will be combined.

In [ ]:
df_lipinski.shape

In [ ]:
df

Now, let's combine the 2 DataFrame

In [ ]:
df_combined = pd.concat([df,df_lipinski], axis=1)

In [ ]:
df_combined

### **Convert IC50 to pIC50**
To allow **IC50** data to be more uniformly distributed, we will convert **IC50** to the negative logarithmic scale which is essentially **-log10(IC50)**.

This custom function pIC50() will accept a DataFrame as input and will:
* Take the IC50 values from the ``standard_value`` column and converts it from nM to M by multiplying the value by 10$^{-9}$
* Take the molar value and apply -log10
* Delete the ``standard_value`` column and create a new ``pIC50`` column

In [ ]:
# https://github.com/chaninlab/estrogen-receptor-alpha-qsar/blob/master/02_ER_alpha_RO5.ipynb

import numpy as np

def pIC50(input):
    pIC50 = []

    for i in input['standard_value_norm']:
        molar = i*(10**-9) # Converts nM to M
        pIC50.append(-np.log10(molar))

    input['pIC50'] = pIC50
    x = input.drop('standard_value_norm',axis= 1)

    return x

Point to note: Values greater than 100,000,000 will be fixed at 100,000,000 otherwise the negative logarithmic value will become negative.

In [ ]:
df_combined.standard_value.describe()

In [ ]:
-np.log10( (10**-9)* 100000000 )

In [ ]:
-np.log10( (10**-9)* 10000000000 )

In [ ]:
def norm_value(input):
    norm = []

    for i in input['standard_value']:
        if i > 100000000:
          i = 100000000
        norm.append(i)

    input['standard_value_norm'] = norm
    x = input.drop('standard_value', axis=1)

    return x

We will first apply the norm_value() function so that the values in the standard_value column is normalized.

In [ ]:
df_norm = norm_value(df_combined)
df_norm

In [ ]:
df_norm.standard_value_norm.describe()

In [ ]:
df_final = pIC50(df_norm)
df_final

In [ ]:
df_final.pIC50.describe()

### **Removing the 'intermediate' bioactivity class**
Here, we will be removing the ``intermediate`` class from our data set.

In [ ]:
df_2class = df_final[df_final.bioactivity_class != 'intermediate']
df_2class

In [ ]:
df_2class.shape

---

## **Exploratory Data Analysis (Chemical Space Analysis) via Lipinski descriptors**

### **Import library**

In [ ]:
import seaborn as sns
sns.set(style='ticks')
import matplotlib.pyplot as plt

### **Frequency plot of the 2 bioactivity classes**

In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.countplot(x='bioactivity_class', data=df_2class, edgecolor='black')

plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('Frequency', fontsize=14, fontweight='bold')

plt.savefig('plot_bioactivity_class.pdf')

### **Scatter plot of MW versus LogP**

It can be seen that the 2 bioactivity classes are spanning similar chemical spaces as evident by the scatter plot of MW vs LogP.

In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.scatterplot(x='MW', y='LogP', data=df_2class, hue='bioactivity_class', size='pIC50', edgecolor='black', alpha=0.7)

plt.xlabel('MW', fontsize=14, fontweight='bold')
plt.ylabel('LogP', fontsize=14, fontweight='bold')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0)
plt.savefig('plot_MW_vs_LogP.pdf')

### **Box plots**

#### **pIC50 value**

In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.boxplot(x = 'bioactivity_class', y = 'pIC50', data = df_2class)

plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('pIC50 value', fontsize=14, fontweight='bold')

plt.savefig('plot_ic50.pdf')

**Statistical analysis | Mann-Whitney U Test**

In [ ]:
def mannwhitney(descriptor, verbose=False):
  # https://machinelearningmastery.com/nonparametric-statistical-significance-tests-in-python/
  from numpy.random import seed
  from numpy.random import randn
  from scipy.stats import mannwhitneyu

# seed the random number generator
  seed(1)

# actives and inactives
  selection = [descriptor, 'bioactivity_class']
  df = df_2class[selection]
  active = df[df.bioactivity_class == 'active']
  active = active[descriptor]

  selection = [descriptor, 'bioactivity_class']
  df = df_2class[selection]
  inactive = df[df.bioactivity_class == 'inactive']
  inactive = inactive[descriptor]

# compare samples
  stat, p = mannwhitneyu(active, inactive)
  #print('Statistics=%.3f, p=%.3f' % (stat, p))

# interpret
  alpha = 0.05
  if p > alpha:
    interpretation = 'Same distribution (fail to reject H0)'
  else:
    interpretation = 'Different distribution (reject H0)'

  results = pd.DataFrame({'Descriptor':descriptor,
                          'Statistics':stat,
                          'p':p,
                          'alpha':alpha,
                          'Interpretation':interpretation}, index=[0])
  filename = 'mannwhitneyu_' + descriptor + '.csv'
  results.to_csv(filename)

  return results

In [ ]:
mannwhitney('pIC50')

#### **MW**

In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.boxplot(x = 'bioactivity_class', y = 'MW', data = df_2class)

plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('MW', fontsize=14, fontweight='bold')

plt.savefig('plot_MW.pdf')

In [ ]:
mannwhitney('MW')

#### **LogP**

In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.boxplot(x = 'bioactivity_class', y = 'LogP', data = df_2class)

plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('LogP', fontsize=14, fontweight='bold')

plt.savefig('plot_LogP.pdf')

**Statistical analysis | Mann-Whitney U Test**

In [ ]:
mannwhitney('LogP')

#### **NumHDonors**

In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.boxplot(x = 'bioactivity_class', y = 'NumHDonors', data = df_2class)

plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('NumHDonors', fontsize=14, fontweight='bold')

plt.savefig('plot_NumHDonors.pdf')

**Statistical analysis | Mann-Whitney U Test**

In [ ]:
mannwhitney('NumHDonors')

#### **NumHAcceptors**

In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.boxplot(x = 'bioactivity_class', y = 'NumHAcceptors', data = df_2class)

plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('NumHAcceptors', fontsize=14, fontweight='bold')

plt.savefig('plot_NumHAcceptors.pdf')

In [ ]:
mannwhitney('NumHAcceptors')

#### **Interpretation of Statistical Results**

##### **Box Plots**

###### **pIC50 values**

Taking a look at pIC50 values, the **actives** and **inactives** displayed ***statistically significant difference***, which is to be expected since threshold values (``IC50 < 1,000 nM = Actives while IC50 > 10,000 nM = Inactives``, corresponding to ``pIC50 > 6 = Actives and pIC50 < 5 = Inactives``) were used to define actives and inactives.

###### **Lipinski's descriptors**

Of the 4 Lipinski's descriptors (MW, LogP, NumHDonors and NumHAcceptors), only LogP exhibited ***no difference*** between the **actives** and **inactives** while the other 3 descriptors (MW, NumHDonors and NumHAcceptors) shows ***statistically significant difference*** between **actives** and **inactives**.

## **Zip files**

In [ ]:
! zip -r results.zip . -i *.csv *.pdf

## **Download PaDEL-Descriptor**

In [ ]:
! wget https://github.com/dataprofessor/bioinformatics/raw/master/padel.zip
! wget https://github.com/dataprofessor/bioinformatics/raw/master/padel.sh

In [ ]:
! unzip padel.zip

## **Load bioactivity data**

Download the curated ChEMBL bioactivity data that has been pre-processed from Parts 1 and 2 of this Bioinformatics Project series. Here we will be using the **bioactivity_data_3class_pIC50.csv** file that essentially contain the pIC50 values that we will be using for building a regression model.

In [ ]:
! wget https://raw.githubusercontent.com/dataprofessor/data/master/acetylcholinesterase_04_bioactivity_data_3class_pIC50.csv

In [ ]:
import pandas as pd

In [ ]:
df3 = pd.read_csv('acetylcholinesterase_04_bioactivity_data_3class_pIC50.csv')

In [ ]:
df3

In [ ]:
selection = ['canonical_smiles','molecule_chembl_id']
df3_selection = df3[selection]
df3_selection.to_csv('molecule.smi', sep='\t', index=False, header=False)

In [ ]:
! cat molecule.smi | head -5

In [ ]:
! cat molecule.smi | wc -l

## **Calculate fingerprint descriptors**


### **Calculate PaDEL descriptors**

In [ ]:
! cat padel.sh

In [ ]:
! bash padel.sh

In [ ]:
! ls -l

## **Preparing the X and Y Data Matrices**

### **X data matrix**

In [ ]:
df3_X = pd.read_csv('descriptors_output.csv')

In [ ]:
df3_X

In [ ]:
df3_X = df3_X.drop(columns=['Name'])
df3_X

## **Y variable**

### **Convert IC50 to pIC50**

In [ ]:
df3_Y = df3['pIC50']
df3_Y

## **Combining X and Y variable**

In [ ]:
dataset3 = pd.concat([df3_X,df3_Y], axis=1)
dataset3

In [ ]:
dataset3.to_csv('acetylcholinesterase_06_bioactivity_data_3class_pIC50_pubchem_fp.csv', index=False)

# **Let's download the CSV file to your local computer for the Part 3B (Model Building).**

## **1. Import libraries**

In [ ]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

## **2. Load the data set**

In [ ]:
! wget https://github.com/dataprofessor/data/raw/master/acetylcholinesterase_06_bioactivity_data_3class_pIC50_pubchem_fp.csv

In [ ]:
df = pd.read_csv('acetylcholinesterase_06_bioactivity_data_3class_pIC50_pubchem_fp.csv')

## **3. Input features**
The ***Acetylcholinesterase*** data set contains 881 input features and 1 output variable (pIC50 values).

### **3.1. Input features**

In [ ]:
X = df.drop('pIC50', axis=1)
X

### **3.2. Output features**

In [ ]:
Y = df.pIC50
Y

### **3.3. Let's examine the data dimension**

In [ ]:
X.shape

In [ ]:
Y.shape

### **3.4. Remove low variance features**

In [ ]:
from sklearn.feature_selection import VarianceThreshold
selection = VarianceThreshold(threshold=(.8 * (1 - .8)))
X = selection.fit_transform(X)

In [ ]:
X.shape

## **4. Data split (80/20 ratio)**

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [ ]:
X_train.shape, Y_train.shape

In [ ]:
X_test.shape, Y_test.shape

## **5. Building a Regression Model using Random Forest**

In [ ]:
model = RandomForestRegressor(n_estimators=100)
model.fit(X_train, Y_train)
r2 = model.score(X_test, Y_test)
r2

In [ ]:
Y_pred = model.predict(X_test)

## **6. Scatter Plot of Experimental vs Predicted pIC50 Values**

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(color_codes=True)
sns.set_style("white")

ax = sns.regplot(x=Y_test, y=Y_pred, scatter_kws={'alpha': 0.4})
ax.set_xlabel('Experimental pIC50', fontsize='large', fontweight='bold')
ax.set_ylabel('Predicted pIC50', fontsize='large', fontweight='bold')
ax.set_xlim(0, 12)
ax.set_ylim(0, 12)
ax.figure.set_size_inches(5, 5)

plt.show()
